<a href="https://colab.research.google.com/github/racoope70/daytrading-with-ml/blob/main/multi_stock_ppo_inference_enhanced.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install yfinance pywavelets transformers --upgrade

In [ ]:
!apt-get remove --purge -y cuda* libcuda* nvidia* || echo "No conflicting CUDA packages"
!apt-get autoremove -y
!apt-get clean

In [ ]:

!apt-get update -qq && apt-get install -y     libcusover11 libcusparse11 libcurand10 libcufft10 libnppig10 ibnppc10 libnppial10
   cuda-toolkit-12-4


In [ ]:
!pip uninstall -y protobuf
!pip install protobuf==3.20.3


In [ ]:
!pip install --extra-index-url=https://pypi.nvidia.com
    cuml-cu12==25.2.0 cudf-cu12==25.2.0 cupy-cuda12x
    dask-cuda==25.2.0 dask-cudf-cu12==25.2.0


In [ ]:
!pip install numba==0.60.0


In [ ]:
!pip install stable-baselines3[extra] gymnasium gym-anytrading yfinance xgboost joblib
!pip install matplotlib scikit-learn pandas


In [ ]:
!pip install tensorflow==2.18.0


In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124


In [ ]:
import tensorflow as tf

gpus = tf.config.list_physical_devices("GPU")
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("TensorFlow GPU memory growth enabled")
    except RuntimeError as e:
        print(f"TensorFlow GPU memory config failed: {e}")


In [ ]:
import os
os.environ['CUDA_HOME'] = '/usr/local/cuda-12.4'
os.environ['PATH'] += ':/usr/local/cuda-12.4/bin'
os.environ['LD_LIBRARY_PATH'] += ':/usr/local/cuda-12.4/lib64'


In [ ]:
!rm -rf /content/drive

In [ ]:
#Step 7: authenticate with hugging face hub (optional)
#This allows for better access and avoids rate limits when downloading public models/datasets

# Authenticate with Hugging Face Hub
#notebook_login()

In [ ]:
# === Download + Feature Engineering (standalone) ===
# --- Imports ---
import os, gc, time, json, pywt, logging
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import yfinance as yf
from google.colab import drive

# ---------------- Logging ----------------
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# ---------------- Google Drive ----------------
drive.mount('/content/drive')

USE_REGIME = True  # toggle


# ---------------- Config ----------------
ticker_list = [
    'AAPL','TSLA','MSFT','GOOGL','AMZN','NVDA','META','BRK-B','JPM','JNJ',
    'XOM','V','PG','UNH','MA','HD','LLY','MRK','PEP','KO',
    'BAC','ABBV','AVGO','PFE','COST','CSCO','TMO','ABT','ACN','WMT',
    'MCD','ADBE','DHR','CRM','NKE','INTC','QCOM','NEE','AMD','TXN',
    'AMGN','UPS','LIN','PM','UNP','BMY','LOW','RTX','CVX','IBM',
    'GE','SBUX','ORCL'
]
strategy_name = "ppo_walkforward_v1"

TEST_MODE = True  # set False to process full list
SYMBOLS = ['AAPL', 'NVDA', 'MSFT'] if TEST_MODE else ticker_list

# Data horizon
end_date = datetime.today()
start_date = end_date - timedelta(days=730)
INTERVAL = "1h"

# Output paths
LOCAL_OUT = "multi_stock_feature_engineered_dataset.csv"
DRIVE_DIR = "/content/drive/MyDrive/trading_data/"
os.makedirs(DRIVE_DIR, exist_ok=True)

# ---------------- Sentiment Toggle ----------------
USE_SENTIMENT = False  # set True to enable FinBERT

if USE_SENTIMENT:
    from transformers import pipeline
    try:
        import torch
        device_id = 0 if torch.cuda.is_available() else -1
    except Exception:
        device_id = -1
    sentiment_pipeline = pipeline(
        "sentiment-analysis",
        model="ProsusAI/finbert",
        device=device_id
    )
else:
    sentiment_pipeline = None  # ensures code path stays simple when off

# ---------------- Helpers ----------------
def download_stock_data(ticker, start_date=None, end_date=None, interval="1h",
                        max_retries=5, window_days=730):
    """Download OHLCV from yfinance with retries."""
    if start_date is None or end_date is None:
        end_date = datetime.today()
        start_date = end_date - timedelta(days=window_days)
    start_str = start_date.strftime('%Y-%m-%d')
    end_str = end_date.strftime('%Y-%m-%d')

    for attempt in range(1, max_retries + 1):
        try:
            logging.info(f"📥 Attempt {attempt}: {ticker} {interval} {start_str}→{end_str}")
            df = yf.download(
                ticker,
                start=start_str,
                end=end_str,
                interval=interval,
                progress=False,
                auto_adjust=False
            )
            if not df.empty:
                df = df.copy()
                df.reset_index(inplace=True)
                df['Symbol'] = ticker
                # yfinance uses 'Datetime' for intraday, 'Date' for daily
                dt_col = 'Datetime' if 'Datetime' in df.columns else 'Date'
                df['Datetime'] = pd.to_datetime(df[dt_col])

                # ensure unique, sorted timestamps
                df = (
                    df.drop_duplicates(subset=['Datetime'])
                      .sort_values('Datetime')
                      .reset_index(drop=True)
                )
                return df
            else:
                raise ValueError("Empty data")
        except Exception as e:
            wait = attempt * 5
            logging.warning(f"⚠️ {ticker} download error: {e} | retrying in {wait}s")
            time.sleep(wait)

    logging.error(f"❌ Failed to download {ticker}")
    return None

def denoise_wavelet(series, wavelet='db1', level=2):
    s = pd.Series(series).astype(float).ffill().bfill().to_numpy()
    try:
        coeffs = pywt.wavedec(s, wavelet, mode='symmetric', level=level)
        for i in range(1, len(coeffs)):
            coeffs[i] = np.zeros_like(coeffs[i])
        rec = pywt.waverec(coeffs, wavelet, mode='symmetric')
        return pd.Series(rec[:len(s)], index=series.index)
    except Exception as e:
        logging.warning(f"Wavelet denoising failed ({e}); using raw Close.")
        return pd.Series(s, index=series.index)

def score_sentiment(texts):
    """Return sentiment scores aligned to texts; neutral when disabled."""
    if not USE_SENTIMENT or sentiment_pipeline is None:
        return [0] * len(texts)
    try:
        outputs = sentiment_pipeline(texts, truncation=True, max_length=256, batch_size=64)
        scores = []
        for r in outputs:
            label = r['label'].lower()
            if label == 'positive':
                scores.append(+float(r['score']))
            elif label == 'negative':
                scores.append(-float(r['score']))
            else:  # neutral
                scores.append(0.0)
        return scores
    except Exception as e:
        logging.error(f"Sentiment scoring error: {e}")
        return [0] * len(texts)
def add_regime(df: pd.DataFrame) -> pd.DataFrame:
    # 2 fast, robust signals: rolling vol & 20‑bar return
    df['Vol20'] = df['Close'].pct_change().rolling(20).std()
    df['Ret20'] = df['Close'].pct_change(20)
    # 4-bin regime: (high/low vol) x (trending/ranging)
    vol_hi   = (df['Vol20'] > df['Vol20'].median()).astype(int)
    trend_hi = (df['Ret20'].abs() > df['Ret20'].abs().median()).astype(int)
    df['Regime4'] = vol_hi * 2 + trend_hi  # 0..3
    return df

def compute_enhanced_features(df):
    df = df.copy()
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(0)
    df = df.loc[:, ~df.columns.duplicated()]

    # --- Technicals ---
    df['SMA_20'] = df['Close'].rolling(20).mean()
    df['STD_20'] = df['Close'].rolling(20).std()
    df['Upper_Band'] = df['SMA_20'] + 2 * df['STD_20']
    df['Lower_Band'] = df['SMA_20'] - 2 * df['STD_20']

    df['Lowest_Low'] = df['Low'].rolling(14).min()
    df['Highest_High'] = df['High'].rolling(14).max()
    denom = (df['Highest_High'] - df['Lowest_Low']).replace(0, np.nan)
    df['Stoch'] = ((df['Close'] - df['Lowest_Low']) / denom) * 100

    df['ROC'] = df['Close'].pct_change(10)
    df['OBV'] = (np.sign(df['Close'].diff()).fillna(0) * df['Volume'].fillna(0)).cumsum()

    tp = (df['High'] + df['Low'] + df['Close']) / 3
    sma_tp = tp.rolling(20).mean()
    md = (tp - sma_tp).abs().rolling(20).mean()
    df['CCI'] = (tp - sma_tp) / (0.015 * md)

    df['EMA_10'] = df['Close'].ewm(span=10, adjust=False).mean()
    df['EMA_50'] = df['Close'].ewm(span=50, adjust=False).mean()
    ema12 = df['Close'].ewm(span=12, adjust=False).mean()
    ema26 = df['Close'].ewm(span=26, adjust=False).mean()
    df['MACD_Line'] = ema12 - ema26
    df['MACD_Signal'] = df['MACD_Line'].ewm(span=9, adjust=False).mean()

    delta = df['Close'].diff()
    gain = delta.clip(lower=0).rolling(14).mean()
    loss = (-delta.clip(upper=0)).rolling(14).mean()
    rs = gain / (loss.replace(0, np.nan))
    df['RSI'] = 100 - (100 / (1 + rs))

    tr = pd.concat([
        (df['High'] - df['Low']),
        (df['High'] - df['Close'].shift()).abs(),
        (df['Low'] - df['Close'].shift()).abs()
    ], axis=1).max(axis=1)
    df['ATR'] = tr.rolling(14).mean()

    df['Volatility'] = df['Close'].pct_change().rolling(20).std()

    df['Return'] = (df['Close'].shift(-10) - df['Close']) / df['Close']
    df['Target'] = np.select([df['Return'] > 0.02, df['Return'] < -0.02], [1, -1], default=0)

    # --- Denoised Close ---
    df['Denoised_Close'] = denoise_wavelet(df['Close'].ffill())
    # --- Optional market regime feature ---
    if USE_REGIME:
        df = add_regime(df)

    # --- Sentiment (toggle) ---
    headline = f"{df['Symbol'].iloc[0]} is expected to perform well in the market." if len(df) else ""
    df['Mock_Headline'] = headline
    try:
        single_score = score_sentiment([headline])[0] if (USE_SENTIMENT and len(df)) else 0.0
    except Exception as e:
        logging.warning(f"Sentiment scoring failed for {df['Symbol'].iloc[0] if 'Symbol' in df else '?'}: {e}")
        single_score = 0.0
    df['SentimentScore'] = float(single_score)
    df.drop(columns=['Mock_Headline'], errors='ignore', inplace=True)

# inside compute_enhanced_features(...)
if USE_REGIME:
    df = add_regime(df)

    # --- Greeks-ish (simple proxies) ---
    df['Delta'] = df['Close'].pct_change(1).fillna(0)
    df['Gamma'] = df['Delta'].diff().fillna(0)

    # Final cleanup
    df = df.dropna().reset_index(drop=True)

    cols = [c for c in df.columns if c not in ['Target', 'Return', 'Symbol']] + ['Target', 'Return', 'Symbol']
    return df[cols]

# ---------------- Main Loop ----------------
all_dfs = []
for i, ticker in enumerate(SYMBOLS, 1):
    logging.info(f"[{i}/{len(SYMBOLS)}] Processing {ticker}")
    raw = download_stock_data(ticker, start_date, end_date, interval=INTERVAL)
    if raw is None or raw.empty:
        logging.warning(f"No data for {ticker}")
        continue

    try:
        features = compute_enhanced_features(raw)
        if features is not None and not features.empty:
            all_dfs.append(features)
        else:
            logging.warning(f"Feature set empty for {ticker}")
    except Exception as e:
        logging.error(f"Feature engineering failed for {ticker}: {e}")
    finally:
        del raw
        try:
            del features
        except NameError:
            pass
        gc.collect()

# ---------------- Save ----------------
if all_dfs:
    final_df = pd.concat(all_dfs, ignore_index=True)
    logging.info(f"✅ Combined dataset shape: {final_df.shape}")

    # save locally
    final_df.to_csv(LOCAL_OUT, index=False)
    logging.info(f"Saved local CSV: {LOCAL_OUT}")

    # save to Drive
    drive_csv = os.path.join(DRIVE_DIR, "multi_stock_feature_engineered_dataset.csv")
    final_df.to_csv(drive_csv, index=False)
    logging.info(f"Saved to Google Drive: {drive_csv}")

    # free memory
    del all_dfs, final_df
    gc.collect()
else:
    logging.warning("No usable data found for any ticker.")


In [ ]:
df = pd.read_csv("multi_stock_feature_engineered_dataset.csv")
print(df.head())


In [ ]:
# === PPO Walkforward with Runtime Timing, Full Model Saving, and Logging ===
import os, gc, time, json, torch, logging, joblib
import pandas as pd, numpy as np, matplotlib.pyplot as plt
import heapq
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from gym_anytrading.envs import StocksEnv
import gymnasium as gym
from gymnasium.spaces import Box
from stable_baselines3.common.utils import set_random_seed

set_random_seed(42)

# === Configuration ===

RESULTS_DIR = "/content/drive/MyDrive/Results_May_2025/ppo_walkforward_results"
FINAL_MODEL_DIR = os.path.join(RESULTS_DIR, "models")
os.makedirs(RESULTS_DIR, exist_ok=True)
os.makedirs(FINAL_MODEL_DIR, exist_ok=True)

# === Logging Setup ===

logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

# === Flags ===

ENABLE_SENTIMENT = False
ENABLE_SLO = True
ENABLE_WAVELET = True
test_mode = True
ENABLE_PLOTS = False
LIVE_MODE = False           # simple live/paper inference loop (set True to run)
SIM_LATENCY_MS = 150        # network/broker latency simulation; 0 = off
BROKER = "log"              # "log" = do not place orders, just log


# === Load Dataset ===

if not os.path.exists("multi_stock_feature_engineered_dataset.csv"):
raise FileNotFoundError("Required feature-engineered dataset not found!")
df = pd.read_csv("multi_stock_feature_engineered_dataset.csv")
df['Datetime'] = pd.to_datetime(df['Datetime'])

# === PPO-Compatible Environment ===
class ContinuousTradingEnv(StocksEnv):
    def __init__(self, df, frame_bound, window_size):
        super().__init__(df=df.reset_index(drop=True), frame_bound=frame_bound, window_size=window_size)
        self.action_space = Box(low=-1.0, high=1.0, shape=(1,), dtype=np.float32)

    def step(self, action):
        try:
            # ---- map continuous -> discrete ----
            a = float(np.array(action).squeeze())
            if a < -0.3:
                discrete_action = 0   # sell
            elif a > 0.1:
                discrete_action = 1   # buy
            else:
                discrete_action = 2   # hold

            # parent step (gym-anytrading may return 4- or 5-tuple)
            step_result = super().step(discrete_action)
            if len(step_result) == 5:
                obs, reward, terminated, truncated, info = step_result
                done = bool(terminated or truncated)
            else:
                obs, reward, done, info = step_result

            # ---- execution pricing (with optional slippage) ----
            cur  = self.df.loc[self._current_tick, 'Close']
            prev = self.df.loc[max(self._current_tick - 1, 0), 'Close']

            slippage_pct = 0.002 if ENABLE_SLO else 0.0
            if discrete_action == 1:       # buy
                fill = cur * (1 + slippage_pct)
            elif discrete_action == 0:     # sell
                fill = cur * (1 - slippage_pct)
            else:                          # hold
                fill = cur

            # base PnL shaping (directional)
            reward += (fill - prev) * (1 if discrete_action == 1 else (-1 if discrete_action == 0 else 0))
            # small encouragement for decisive actions
            reward += abs(a) * 0.01

            # optional wavelet trend shaping (tiny nudge)
            if ENABLE_WAVELET and 'Denoised_Close' in self.df.columns:
                prev_idx = max(self._current_tick - 1, 0)
                dn_delta = float(self.df.loc[self._current_tick, 'Denoised_Close'] -
                                 self.df.loc[prev_idx, 'Denoised_Close'])
                reward += 0.001 * np.tanh(dn_delta)

            # optional sentiment shaping
            if ENABLE_SENTIMENT and 'SentimentScore' in self.df.columns:
                sent = float(self.df.loc[self._current_tick, 'SentimentScore'])
                reward += sent * 0.01

            # final clip + return 4-tuple for SB3 VecEnv
            return obs, float(np.clip(reward, -1.0, 1.0)), done, info

        except Exception as e:
            logging.error(f"Step error: {e}")
            try:
                obs = self.reset()
            except Exception:
                obs = super().reset() if hasattr(super(), "reset") else None
            return obs, 0.0, True, {}

# === Utilities ===

def plot_performance(ticker, portfolio, hold_value):
    plt.figure(figsize=(10, 4))
    plt.plot(portfolio, label="PPO Portfolio")
    plt.axhline(y=hold_value, color="r", linestyle="--", label="Buy & Hold")
    plt.title(f"{ticker} - PPO vs Buy & Hold")
    plt.legend()
    plt.grid(True, linestyle="--", alpha=0.5)
    plt.tight_layout()
    plt.savefig(os.path.join(RESULTS_DIR, f"{ticker}_performance.png"))
    plt.close()

def get_walk_forward_windows(df, window_size=3000, step_size=500, min_len=1000):
    return [
        (start, start + window_size)
        for start in range(0, len(df) - min_len, step_size)
        if start + window_size < len(df)
    ]

# === Save QuantConnect-Compatible Artifacts (fixed) ===

def save_quantconnect_model(artifact, prefix, save_dir):
    model_obj = artifact.get("model", None)
    model_path = os.path.join(save_dir, f"{prefix}_model.zip")
    if model_obj is not None and not os.path.exists(model_path):
        model_obj.save(model_path)

    vecnorm_src = artifact.get("vecnorm_path")
    if vecnorm_src:
        try:
            vecnorm_dst = os.path.join(save_dir, f"{prefix}_vecnorm.pkl")
            if os.path.abspath(vecnorm_src) != os.path.abspath(vecnorm_dst):
                import shutil
                shutil.copyfile(vecnorm_src, vecnorm_dst)
        except Exception as e:
            logging.warning(f"VecNormalize file handling issue for {prefix}: {e}")

    with open(os.path.join(save_dir, f"{prefix}_features.json"), "w") as f:
        json.dump({"features": artifact['features']}, f)

    with open(os.path.join(save_dir, f"{prefix}_probability_config.json"), "w") as f:
        json.dump({"threshold": 0.05, "use_confidence": True, "inference_mode": "deterministic"}, f)

    with open(os.path.join(save_dir, f"{prefix}_model_info.json"), "w") as f:
        json.dump({
            "model": "PPO",
            "ticker": artifact['result']['Ticker'],
            "window": artifact['result']['Window'],
            "date_trained": datetime.today().strftime("%Y-%m-%d"),
            "framework": "stable-baselines3",
            "input_features": artifact['features'],
            "final_portfolio": artifact['result']['PPO_Portfolio'],
            "buy_hold": artifact['result']['BuyHold'],
            "sharpe": artifact['result']['Sharpe']
        }, f)

    logging.info(f"Saved QuantConnect-compatible model for {artifact['result']['Ticker']} | {artifact['result']['Window']}")

# ----- Live inference helpers (uses saved PPO + VecNormalize) -----

def load_model_and_env(prefix):
    """Load a trained PPO and create a factory to build a matching env window."""
    model_path = os.path.join(FINAL_MODEL_DIR, f"{prefix}_model.zip")
    vec_path   = os.path.join(FINAL_MODEL_DIR, f"{prefix}_vecnorm.pkl")
    model = PPO.load(model_path, device="cpu")

    def make_env(df_window):
        frame_bound = (50, len(df_window) - 3)
        e = DummyVecEnv([lambda: ContinuousTradingEnv(
            df=df_window, frame_bound=frame_bound, window_size=10
        )])
        if os.path.exists(vec_path):
            e = VecNormalize.load(vec_path, e)
        # eval mode: do not update running stats
        e.training = False
        e.norm_reward = False
        return e

    return model, make_env

def latest_df_for_symbol(symbol, horizon_days=5, interval="1m"):
    """Fetch fresh bars and rebuild features exactly like training."""
    end = datetime.utcnow()
    start = end - timedelta(days=horizon_days)
    df = yf.download(
        symbol,
        start=start.strftime("%Y-%m-%d"),
        end=end.strftime("%Y-%m-%d"),
        interval=interval,
        progress=False,
        auto_adjust=False
    )
    if df is None or df.empty:
        return None
    df = df.reset_index()
    df['Symbol'] = symbol
    # IMPORTANT: reuse your same feature function
    return compute_enhanced_features(df)

def predict_latest(symbol, prefix):
    """Build last window, fast‑forward env, call model.predict(), return a signal."""
    model, make_env = load_model_and_env(prefix)

    live_df = latest_df_for_symbol(symbol)
    if live_df is None or len(live_df) < 100:
        logging.warning("No fresh data yet for live inference.")
        return None

    df_window = live_df.iloc[-2500:].reset_index(drop=True) if len(live_df) > 2500 else live_df.copy()

    env = make_env(df_window)
    obs = env.reset()
    # fast‑forward to end (no-op actions just to advance)
    for _ in range(len(df_window) - 1):
        obs, _, done, _ = env.step([0.0])
        if isinstance(done, (np.ndarray, list)) and done[0]:
            break

    action, _ = model.predict(obs, deterministic=True)
    a = float(np.array(action).squeeze())
    signal = "BUY" if a > 0.1 else ("SELL" if a < -0.3 else "HOLD")
    conf = abs(a)
    ts = df_window['Datetime'].iloc[-1] if 'Datetime' in df_window.columns else None
    price = float(df_window['Close'].iloc[-1])
    return dict(signal=signal, confidence=conf, action=a, ts=ts, price=price)

def place_order(signal, qty=1):
    """Stub broker router with latency simulation; logs in Colab."""
    if SIM_LATENCY_MS > 0:
        time.sleep(SIM_LATENCY_MS / 1000.0)
    if BROKER == "log":
        logging.info(f"[PAPER] {signal} x{qty}")
    else:
        # Hook up real broker SDK here (Alpaca, IB, etc.)
        logging.info(f"[BROKER={BROKER}] {signal} x{qty} (not implemented)")

def live_loop(symbol, best_prefix):
    """Simple polling loop—set LIVE_MODE=True to run."""
    while LIVE_MODE:
        try:
            pred = predict_latest(symbol, best_prefix)
            if pred:
                logging.info(
                    f"{symbol} {pred['ts']} | {pred['signal']} @ {pred['price']:.2f} (conf {pred['confidence']:.2f})"
                )
                place_order(pred['signal'], qty=1)
        except Exception as e:
            logging.error(f"Live loop error: {e}")
        time.sleep(60)  # poll each minute

TOP_N_WINDOWS = 3

# === PPO Walkforward Function ===

def walkforward_ppo(df, ticker, window_size=2500, step_size=500,
                    timesteps=150_000, learning_rate=2e-5, ppo_overrides=None):
    if ppo_overrides is None:
        ppo_overrides = {}
    if len(df) < window_size:
        logging.warning(f"Skipping {ticker}: only {len(df)} rows (min required: {window_size})")
        return []

    results = []
    windows = get_walk_forward_windows(df, window_size, step_size)
    top_heap = []  # (Sharpe, prefix, meta)

    for w_idx, (start, end) in enumerate(windows):
        window_start_time = time.time()
        gc.collect()

        df_window = df.iloc[start:end].reset_index(drop=True)
        if len(df_window) <= 52 or len(df_window) % 2 != 0:
            df_window = df_window.iloc[:-1]

        frame_bound = (50, len(df_window) - 3)
        env = DummyVecEnv([lambda: ContinuousTradingEnv(df=df_window, frame_bound=frame_bound, window_size=10)])
        env = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=10.)

        try:
            model = PPO(
                "MlpPolicy",
                env,
                verbose=0,
                device=("cuda" if torch.cuda.is_available() else "cpu"),
                learning_rate=ppo_overrides.get("lr", learning_rate),
                n_steps=ppo_overrides.get("n_steps", 256),
                batch_size=ppo_overrides.get("batch", 64),
                n_epochs=8,
                gamma=0.99,
                gae_lambda=0.95,
                clip_range=ppo_overrides.get("clip", 0.2),
                ent_coef=ppo_overrides.get("ent", 0.005),
                policy_kwargs=dict(net_arch=[64, 64]),
            )

            logging.info(f"Training {ticker} Window {w_idx+1}/{len(windows)}")
            model.learn(total_timesteps=timesteps)

            # === Prepare env for evaluation (raw rewards, no running stats updates) ===
            env.training = False
            env.norm_reward = False

            # === Evaluation ===
            obs = env.reset()
            pred_rows = []
            position, balance, portfolio = 0.0, 100000.0, []
            trade_log = []
            entry_price, last_trade_step = None, -10
            confidence_buckets = {'Low': 0, 'Mid': 0, 'High': 0}
            cooldown_skips = []
            TRADE_COOLDOWN_STEPS = max(1, int(0.3 * len(df_window) / 100))

            # define prefix BEFORE we save predictions later
            prefix = f"ppo_{ticker}_window{w_idx+1}"

            for i in range(len(df_window)):
                action, _ = model.predict(obs, deterministic=True)
                obs, reward, done, info = env.step(action)
                if isinstance(done, (np.ndarray, list)):
                    done = bool(done[0])

                a = float(np.array(action).squeeze())
                aa = abs(a)  # <-- add this

                # map to string signal for logging
                if a < -0.3:
                    pred_signal = "SELL"
                elif a > 0.1:
                    pred_signal = "BUY"
                else:
                    pred_signal = "HOLD"
                confidence = aa

                price = float(df_window['Close'].iloc[i])
                STOP_LOSS_PCT, TAKE_PROFIT_PCT = 0.04, 0.12
                prior_position = position

                # stop-loss / take-profit
                if position > 0 and entry_price is not None:
                    change = (price - entry_price) / entry_price
                    if change < -STOP_LOSS_PCT or change > TAKE_PROFIT_PCT:
                        balance = position * price
                        position = 0.0
                        trade_log.append("STOP-LOSS" if change < -STOP_LOSS_PCT else "TAKE-PROFIT")
                        entry_price = None

                # trading logic with cooldown
                if aa > 0.02 and (i - last_trade_step) > TRADE_COOLDOWN_STEPS:
                    if a > 0 and position == 0.0:
                        position = balance / price
                        balance = 0.0
                        entry_price = price
                        trade_log.append("BUY")
                        last_trade_step = i
                    elif a < 0 and position > 0.0:
                        balance = position * price
                        position = 0.0
                        trade_log.append("SELL")
                        last_trade_step = i
                    else:
                        trade_log.append("HOLD")
                else:
                    trade_log.append("HOLD")
                    if (i - last_trade_step) <= TRADE_COOLDOWN_STEPS:
                        cooldown_skips.append(i)

                portfolio.append(balance if balance > 0 else position * price)

                # log this step
                pred_rows.append({
                    "Index": i,
                    "Datetime": df_window["Datetime"].iloc[i] if "Datetime" in df_window.columns else None,
                    "Close": price,
                    "Action": a,
                    "Signal": pred_signal,
                    "Confidence": confidence,
                    "Balance": balance,
                    "Position": position,
                    "PortfolioValue": portfolio[-1],
                    "Reward": float(np.array(reward).squeeze())
                })

            if not portfolio:
                logging.warning(f"Empty portfolio for {ticker} | Window {start}-{end}")
                continue

            # save step-by-step predictions AFTER the loop (prefix already defined)
            pred_df = pd.DataFrame(pred_rows)
            pred_path = os.path.join(RESULTS_DIR, f"{prefix}_predictions.csv")
            pred_df.to_csv(pred_path, index=False)
            logging.info(f"Saved predictions to {pred_path}")

            # === Metrics (unchanged) ===
            final_value = float(portfolio[-1])
            hold_value  = float((100000.0 / df_window['Close'].iloc[0]) * df_window['Close'].iloc[-1])
            returns = pd.Series(portfolio).pct_change().fillna(0)
            sharpe  = float((returns.mean() / (returns.std() + 1e-6)) * np.sqrt(252))
            drawdown = float(((pd.Series(portfolio).cummax() - pd.Series(portfolio)) /
                              pd.Series(portfolio).cummax()).max() * 100)

            prefix = f"ppo_{ticker}_window{w_idx+1}"

            # Save VecNormalize snapshot
            vecnorm_path = os.path.join(FINAL_MODEL_DIR, f"{prefix}_vecnorm.pkl")
            try:
                env.save(vecnorm_path)
            except Exception as e:
                logging.warning(f"Could not save VecNormalize for {ticker} {start}-{end}: {e}")
                vecnorm_path = None

            # Persist the model immediately (we may discard later if not top-N)
            model_path = os.path.join(FINAL_MODEL_DIR, f"{prefix}_model.zip")
            model.save(model_path)

            result_row = {
                "Ticker": ticker,
                "Window": f"{start}-{end}",
                "PPO_Portfolio": round(final_value, 2),
                "BuyHold": round(hold_value, 2),
                "Sharpe": round(sharpe, 3),
                "Drawdown_%": round(drawdown, 2),
                "Winner": "PPO" if final_value > hold_value else "Buy & Hold"
            }
            results.append(result_row)

            meta = {
                "result": result_row,
                "features": df_window.columns.tolist(),
                "prefix": prefix,
                "model_path": model_path,
                "vecnorm_path": vecnorm_path
            }

            # Maintain a top-N heap by Sharpe
            item = (result_row["Sharpe"], prefix, meta)
            if len(top_heap) < TOP_N_WINDOWS:
                heapq.heappush(top_heap, item)
            else:
                if item[0] > top_heap[0][0]:
                    _, _, old_meta = heapq.heappop(top_heap)
                    try:
                        if os.path.exists(old_meta["model_path"]): os.remove(old_meta["model_path"])
                        if old_meta["vecnorm_path"] and os.path.exists(old_meta["vecnorm_path"]):
                            os.remove(old_meta["vecnorm_path"])
                    except Exception as e:
                        logging.warning(f"Cleanup of old top-N files failed: {e}")
                    heapq.heappush(top_heap, item)

            # Diagnostics (once)
            logging.info(f"Confidence Buckets for {ticker} | Window {start}-{end}: {confidence_buckets}")
            successful_trades = sum(t == 'TAKE-PROFIT' for t in trade_log)
            total_trades = sum(t in ['SELL', 'STOP-LOSS', 'TAKE-PROFIT'] for t in trade_log)
            hit_ratio = successful_trades / (total_trades + 1e-6)
            logging.info(f"Hit Ratio: {hit_ratio:.2%}")
            if cooldown_skips:
                logging.info(f"Cooldown skips: {len(cooldown_skips)} | Sample: {cooldown_skips[:5]}")
            logging.info(f"{ticker} | Window {w_idx+1} runtime: {round(time.time()-window_start_time, 2)}s")

        finally:
            try: env.close()
            except Exception: pass
            del env
            try: del model
            except Exception: pass
            gc.collect()
            try: torch.cuda.empty_cache()
            except Exception: pass

    # Save top-N (artifact wrapper reads from paths)
    top_list = sorted(top_heap, key=lambda t: t[0], reverse=True)
    for _, _, meta in top_list:
        artifact_for_save = {
            "model": None,  # already on disk
            "vecnorm_path": meta["vecnorm_path"],
            "features": meta["features"],
            "result": meta["result"],
            "prefix": meta["prefix"]
        }
        save_quantconnect_model(artifact_for_save, meta["prefix"], FINAL_MODEL_DIR)

    return results

# --- Per-bucket PPO configs ---

FAST = {"lr": 5e-5, "n_steps": 1024, "batch": 128, "clip": 0.25, "ent": 0.015}
SLOW = {"lr": 1.5e-5, "n_steps": 2048, "batch": 64, "clip": 0.16, "ent": 0.0075}

# Tickers (strings!)

fast_names = {
"TSLA","NVDA","AMD","AVGO","AAPL","MSFT","AMZN","GOOGL","META","ADBE","CRM",
"INTC","QCOM","TXN","ORCL","NEE","GE","XOM","CVX","LLY","NKE","SBUX"
}
slow_names = {
"BRK-B","JPM","BAC","JNJ","UNH","MRK","PFE","ABBV","ABT","AMGN","PG","PEP","KO",
"V","MA","WMT","MCD","TMO","DHR","ACN","IBM","LIN","PM","RTX","UPS","UNP","COST","HD","LOW"
}

def pick_params(symbol: str):
    return FAST if symbol in fast_names else SLOW

def process_ticker(ticker):
    hp = pick_params(ticker)
    return walkforward_ppo(
        df[df['Symbol'] == ticker].copy(),
        ticker,
        window_size=2500,
        timesteps=150_000,
        learning_rate=hp["lr"],
        ppo_overrides=hp
    )

def run_parallel_tickers(tickers, out_path=os.path.join(RESULTS_DIR, "summary.csv"), max_workers=6):
    results = []
    with ThreadPoolExecutor(max_workers=max_workers) as ex:
        for res in ex.map(process_ticker, tickers):
            if res:
                results.extend(res)
                pd.DataFrame(results).to_csv(out_path, index=False)
    logging.info("All tickers processed.")
    return results

# === Execution Block ===

if __name__ == "__main__":
    CONFIG = {'symbols': df['Symbol'].unique().tolist()}
    all_results = []

    if test_mode:
        test_stocks = ['AAPL', 'NVDA', 'MSFT']
        for stock in test_stocks:
            logging.info(f">>> Running test_mode on {stock}")
            results = process_ticker(stock)
            all_results.extend(results)

        test_summary_path = os.path.join(RESULTS_DIR, "summary_test_mode.csv")
        pd.DataFrame(all_results).to_csv(test_summary_path, index=False)
        logging.info(f"Test summary saved to {test_summary_path}")
    else:
        summary_results = run_parallel_tickers(CONFIG['symbols'])
        if not summary_results:
            logging.warning("No results generated.")
        else:
            path = os.path.join(RESULTS_DIR, "summary.csv")
            pd.DataFrame(summary_results).to_csv(path, index=False)
            logging.info(f"Summary saved to {path}")

    logging.info("Script finished execution.")
    gc.collect()
    try:
        torch.cuda.empty_cache()
    except Exception:
        pass
    # --- Optional simple live inference loop ---
    # Pick the best saved prefix for a symbol (e.g., best Sharpe window you kept)
    # Example: best_prefix = "ppo_AAPL_window3"
    best_prefix = None  # <-- set one if you want to run live
    if LIVE_MODE and best_prefix is not None:
        live_loop("AAPL", best_prefix)


In [ ]:
import os
import json
import logging
import joblib
import pandas as pd
from stable_baselines3 import PPO

# === Enable Logging Format ===
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# === Paths ===
RESULTS_DIR = "/content/drive/MyDrive/Results_May_2025/ppo_walkforward_results"
MODEL_DIR = os.path.join(RESULTS_DIR, "models")
TICKER = "AAPL"
WINDOW = "window1"

# === Filenames ===
model_path = os.path.join(MODEL_DIR, f"ppo_{TICKER}_{WINDOW}_model.zip")
vecnorm_path = os.path.join(MODEL_DIR, f"ppo_{TICKER}_{WINDOW}_vecnorm.pkl")
summary_path = os.path.join(RESULTS_DIR, "summary_results.csv")

# === Load PPO Model ===
if os.path.exists(model_path):
    model = PPO.load(model_path)
    logging.info(f"Loaded PPO model from: {model_path}")
else:
    logging.warning(f"Model file not found at: {model_path}")
    model = None

# === Load VecNormalize ===
if os.path.exists(vecnorm_path):
    vec_env = joblib.load(vecnorm_path)
    logging.info(f"Loaded VecNormalize from: {vecnorm_path}")
else:
    logging.warning(f"VecNormalize file not found at: {vecnorm_path}")
    vec_env = None

# === PPO Model Architecture ===
if model:
    logging.info("PPO Model Architecture:")
    logging.info(str(model.policy))

# === VecNormalize Diagnostics ===
if vec_env:
    try:
        logging.info("VecNormalize Statistics:")
        logging.info(f"Mean: {vec_env.obs_rms.mean}")
        logging.info(f"Var: {vec_env.obs_rms.var}")
        logging.info(f"Clip Range: {vec_env.clip_obs}")
    except Exception as e:
        logging.warning(f"Could not extract VecNormalize stats: {e}")

# === Summary Results (Top 3 by Sharpe) ===
if os.path.exists(summary_path):
    try:
        summary_df = pd.read_csv(summary_path)
        top3 = summary_df.sort_values(by="Sharpe", ascending=False).head(3)
        logging.info("Top 3 Results by Sharpe Ratio:")
        for i, row in top3.iterrows():
            logging.info(f"{row['Ticker']} | Window: {row['Window']} | Sharpe: {row['Sharpe']:.3f} | Portfolio: {row['PPO_Portfolio']:.2f} | Drawdown: {row['Drawdown_%']:.2f}%")
    except Exception as e:
        logging.warning(f"Error reading summary file: {e}")
else:
    logging.warning("No summary_results.csv found. Skipping top-3 summary.")
